# Introduction to loading data <img align="right" src="../Supplementary_data/dea_logo.jpg">

* [**Sign up to the DEA Sandbox**](https://docs.dea.ga.gov.au/setup/sandbox.html) to run this notebook interactively from a browser
* **Compatibility:** Notebook currently compatible with both the `NCI` and `DEA Sandbox` environments
* **Products used:** 
[ls7_nbart_geomedian_annual](https://explorer.sandbox.dea.ga.gov.au/ls7_nbart_geomedian_annual),
[ls8_nbart_geomedian_annual](https://explorer.sandbox.dea.ga.gov.au/ls8_nbart_geomedian_annual)
* **Prerequisites:** Users of this notebook should have a basic understanding of:
    * How to run a [Jupyter notebook](01_Jupyter_notebooks.ipynb)
    * The basic structure of the DEA [satellite datasets](02_DEA.ipynb)
    * Inspecting available [DEA products and measurements](03_Products_and_measurements.ipynb)
    

## Background
Loading data from the [Digital Earth Australia (DEA)](https://www.ga.gov.au/dea) instance of the [Open Data Cube](https://www.opendatacube.org/) requires the construction of a data query that specifies the what, where, and when of the data request.
Each query returns a [multi-dimensional xarray object](http://xarray.pydata.org/en/stable/) containing the contents of your query.
It is essential to understand the `xarray` data structures as they are fundamental to the structure of data loaded from the datacube.
Manipulations, transformations and visualisation of `xarray` objects provide datacube users with the ability to explore and analyse DEA datasets, as well as pose and answer scientific questions.

## Description
This notebook will introduce how to load data from the DEA datacube through the construction of a query and use of the `dc.load()` function.
Topics covered include:

1. Loading data using `dc.load()`
2. Interpreting the resulting `xarray.Dataset` object
    * Inspecting an individual `xarray.DataArray`
3. Customising parameters passed to the `dc.load()` function
    * Loading specific measurements
    * Loading data for coordinates in a custom coordinate reference system (CRS)
    * Projecting data to a new CRS and spatial resolution 
    * Specifying a specific spatial resampling method
4. Loading data using a reusable dictionary query
5. Loading matching data from multiple products using `like`
6. Adding a progress bar to the data load

***

## Getting started
To run this introduction to loading data from DEA, run all the cells in the notebook starting with the "Load packages" cell. For help with running notebook cells, refer back to the [Jupyter Notebooks notebook](01_Jupyter_notebooks.ipynb).

### Load packages
The `datacube` package is required to query the datacube database and load some data. 
The `with_ui_cbk` function from `odc.ui` enables a progress bar when loading large amounts of data.

In [1]:
import datacube
from odc.ui import with_ui_cbk

### Connect to the datacube
The next step is to connect to the datacube database.
The resulting `dc` datacube object can then be used to load data.
The `app` parameter is a unique name used to identify the notebook that does not have any effect on the analysis.

In [2]:
dc = datacube.Datacube(app="04_Loading_data")

## Loading data using `dc.load()`

Loading data from the datacube uses the [dc.load()](https://datacube-core.readthedocs.io/en/latest/dev/api/generate/datacube.Datacube.load.html) function.

The function requires the following minimum arguments:

* `product`: The data product to load (to revise DEA products, see the [Products and measurements](03_Products_and_measurements.ipynb) notebook).
* `x`: The spatial region in the *x* dimension. By default, the *x* and *y* arguments accept queries in a geographical co-ordinate system WGS84, identified by the EPSG code *4326*.
* `y`: The spatial region in the *y* dimension. The dimensions ``longitude``/``latitude`` and ``x``/``y`` can be used interchangeably.
* `time`: The temporal extent. The time dimension can be specified using a tuple of datetime objects or strings in the "YYYY", "YYYY-MM" or "YYYY-MM-DD" format. 

For example, to load 2015 data from the [Landsat 8 NBAR-T annual geomedian product](https://explorer.sandbox.dea.ga.gov.au/ls8_nbart_geomedian_annual) for Moreton Bay in southern Queensland, use the following parameters:

* `product`: `ls8_nbart_geomedian_annual`
* `x`: `(153.3, 153.4)`
* `y`: `(-27.5, -27.6)`
* `time`: `("2015-01-01", "2015-12-31")`

Run the following cell to load all datasets from the `ls8_nbart_geomedian_annual` product that match this spatial and temporal extent:

In [3]:
ds = dc.load(product="ls8_nbart_geomedian_annual",
             x=(153.3, 153.4),
             y=(-27.5, -27.6),
             time=("2015-01-01", "2015-12-31"))

print(ds)

<xarray.Dataset>
Dimensions:      (time: 1, x: 461, y: 508)
Coordinates:
  * time         (time) datetime64[ns] 2015-01-01
  * y            (y) float64 -3.156e+06 -3.156e+06 ... -3.168e+06 -3.168e+06
  * x            (x) float64 2.067e+06 2.067e+06 ... 2.079e+06 2.079e+06
    spatial_ref  int32 3577
Data variables:
    blue         (time, y, x) int16 491 490 489 493 499 ... 372 313 289 286 294
    green        (time, y, x) int16 534 538 538 543 548 ... 545 483 421 394 399
    red          (time, y, x) int16 248 252 252 253 257 ... 457 364 347 362 350
    nir          (time, y, x) int16 108 106 105 105 108 ... 2522 2292 2351 2399
    swir1        (time, y, x) int16 64 65 62 62 64 ... 1616 1261 1057 1070 1101
    swir2        (time, y, x) int16 53 53 51 51 54 51 ... 886 627 510 503 512
Attributes:
    crs:           EPSG:3577
    grid_mapping:  spatial_ref


### Interpreting the resulting `xarray.Dataset`
The variable `ds` has returned an `xarray.Dataset` containing all data that matched the spatial and temporal query parameters inputted into `dc.load`.

*Dimensions* 

* This header identifies the number of timesteps returned in the search (`time: 1`) as well as the number of pixels in the `x` and `y` directions of the data query.

*Coordinates* 

* `time` identifies the date attributed to each returned timestep.
* `x` and `y` are the coordinates for each pixel within the spatial bounds of the query.

*Data variables*

* These are the measurements available for the nominated product. 
For every date (`time`) returned by the query, the measured value at each pixel (`y`, `x`) is returned as an array for each measurement.
Each data variable is itself an `xarray.DataArray` object ([see below](#Inspecting-an-individual-xarray.DataArray)). 

*Attributes*

* `crs` identifies the coordinate reference system (CRS) of the loaded data. 

### Inspecting an individual `xarray.DataArray`
The `xarray.Dataset` loaded above is itself a collection of individual `xarray.DataArray` objects that hold the actual data for each data variable/measurement. 
For example, all measurements listed under _Data variables_ above (e.g. `blue`, `green`, `red`, `nir`, `swir1`, `swir2`) are `xarray.DataArray` objects.

These `xarray.DataArray` objects can be inspected or interacted with by using either of the following syntaxes:
```
ds["measurement_name"]
```
or
```
ds.measurement_name
```

The ability to access individual variables means that these can be directly viewed, or further manipulated to create new variables. 
For example, run the following cell to access data from the near infra-red satellite band (i.e. `nir`):

In [4]:
print(ds.nir)

<xarray.DataArray 'nir' (time: 1, y: 508, x: 461)>
array([[[ 108,  106,  105, ...,   95,   95,   98],
        [ 109,  104,  101, ...,   94,   94,  105],
        [ 105,  101,  100, ...,  106,  121,  181],
        ...,
        [3134, 2509, 2417, ..., 2380, 2473, 2366],
        [2678, 3013, 2742, ..., 2183, 2606, 2575],
        [2573, 2948, 2695, ..., 2292, 2351, 2399]]], dtype=int16)
Coordinates:
  * time         (time) datetime64[ns] 2015-01-01
  * y            (y) float64 -3.156e+06 -3.156e+06 ... -3.168e+06 -3.168e+06
  * x            (x) float64 2.067e+06 2.067e+06 ... 2.079e+06 2.079e+06
    spatial_ref  int32 3577
Attributes:
    units:         1
    nodata:        -999
    crs:           EPSG:3577
    grid_mapping:  spatial_ref


Note that the object header informs us that it is an `xarray.DataArray` containing data for the `nir` satellite band. 

Like an `xarray.Dataset`, the array also includes information about the data's **dimensions** (i.e. `(time: 1, y: 508, x: 461)`), **coordinates** and **attributes**.
This particular data variable/measurement contains some additional information that is specific to the `nir` band, including details of array's nodata value (i.e. `nodata: -999`).

> For a more in-depth introduction to `xarray` data structures, refer to the [official xarray documentation](http://xarray.pydata.org/en/stable/data-structures.html)

## Customising the `dc.load()` function

The `dc.load()` function can be tailored to refine a query.

Customisation options include:

* `measurements:` This argument is used to provide a list of measurement names to load, as listed in `dc.list_measurements()`. 
For satellite datasets, measurements contain data for each individual satellite band (e.g. near infrared). 
If not provided, all measurements for the product will be returned.
* `crs:` The coordinate reference system (CRS) of the query's `x` and `y` coordinates is assumed to be `WGS84`/`EPSG:4326` unless the `crs` field is supplied, even if the stored data is in another projection or the `output_crs` is specified. 
The `crs` parameter is required if the query's coordinates are in any other CRS.
* `group_by:` Satellite datasets based around scenes can have multiple observations per day with slightly different time stamps as the satellite collects data along its path.
These observations can be combined by reducing the `time` dimension to the day level using `group_by=solar_day`.
* `output_crs` and `resolution`: To reproject or change the resolution the data, supply the `output_crs` and `resolution` fields.    
* `resampling`: This argument allows you to specify a custom spatial resampling method to use when data is reprojected into a different CRS. 

Example syntax on the use of these options follows in the cells below.

> For help or more customisation options, run `help(dc.load)` in an empty cell or visit the function's [documentation page](https://datacube-core.readthedocs.io/en/latest/dev/api/generate/datacube.Datacube.load.html)


### Specifying measurements
By default, `dc.load()` will load *all* measurements in a product.

To load data from the `red`, `green` and `blue` satellite bands only, add `measurements=["red", "green", "blue"]` to the query:

In [5]:
# Note the optional inclusion of the measurements list
ds_rgb = dc.load(product="ls8_nbart_geomedian_annual",
                 measurements=["red", "green", "blue"],
                 x=(153.3, 153.4),
                 y=(-27.5, -27.6),
                 time=("2015-01-01", "2015-12-31"))

print(ds_rgb)

<xarray.Dataset>
Dimensions:      (time: 1, x: 461, y: 508)
Coordinates:
  * time         (time) datetime64[ns] 2015-01-01
  * y            (y) float64 -3.156e+06 -3.156e+06 ... -3.168e+06 -3.168e+06
  * x            (x) float64 2.067e+06 2.067e+06 ... 2.079e+06 2.079e+06
    spatial_ref  int32 3577
Data variables:
    red          (time, y, x) int16 248 252 252 253 257 ... 457 364 347 362 350
    green        (time, y, x) int16 534 538 538 543 548 ... 545 483 421 394 399
    blue         (time, y, x) int16 491 490 489 493 499 ... 372 313 289 286 294
Attributes:
    crs:           EPSG:3577
    grid_mapping:  spatial_ref


Note that the **Data variables** component of the `xarray.Dataset` now includes only the measurements specified in the query (i.e. the `red`, `green` and `blue` satellite bands).

### Loading data for coordinates in any CRS
By default, `dc.load()` assumes that the queried `x` and `y` coordinates are in the `WGS84`/`EPSG:4326` CRS.
If these coordinates are in a different coordinate system, specify this using the `crs` parameter.

The example cell below loads data for a set of `x` and `y` coordinates defined in Australian Albers (`EPSG:3577`), ensuring that the `dc.load()` function accounts for this by including `crs="EPSG:3577"`:

In [6]:
# Note the new `x` and `y` coordinates and `crs` parameter
ds_custom_crs = dc.load(product="ls8_nbart_geomedian_annual",
                        time=("2015-01-01", "2015-12-31"),
                        x=(2069310, 2077064),
                        y=(-3155823, -3168513),
                        crs="EPSG:3577")

print(ds_custom_crs)

<xarray.Dataset>
Dimensions:      (time: 1, x: 311, y: 509)
Coordinates:
  * time         (time) datetime64[ns] 2015-01-01
  * y            (y) float64 -3.156e+06 -3.156e+06 ... -3.168e+06 -3.169e+06
  * x            (x) float64 2.069e+06 2.069e+06 ... 2.077e+06 2.077e+06
    spatial_ref  int32 3577
Data variables:
    blue         (time, y, x) int16 470 462 464 462 460 ... 438 440 439 431 412
    green        (time, y, x) int16 493 480 481 481 482 ... 497 497 495 484 464
    red          (time, y, x) int16 223 211 212 213 215 ... 261 260 258 245 235
    nir          (time, y, x) int16 87 84 83 82 81 81 ... 117 116 114 110 101
    swir1        (time, y, x) int16 53 47 47 47 46 45 47 ... 66 67 67 67 65 54
    swir2        (time, y, x) int16 44 40 38 38 37 35 38 ... 53 54 54 54 51 42
Attributes:
    crs:           EPSG:3577
    grid_mapping:  spatial_ref


### CRS reprojection
Certain applications may require that data is output into a specific CRS.
Data can be reprojected by specifying the new `output_crs` and identifying the `resolution` required.

The example cell below reprojects data to a new CRS (UTM Zone 56S, `EPSG:32756`) and resolution (250 x 250 m). Note that for most CRSs, the first resolution value is negative (e.g. `(-250, 250)`):

In [7]:
ds_reprojected = dc.load(product="ls8_nbart_geomedian_annual",
                         x=(153.3, 153.4),
                         y=(-27.5, -27.6),
                         time=("2015-01-01", "2015-12-31"),
                         output_crs="EPSG:32756",
                         resolution=(-250, 250))

print(ds_reprojected)

<xarray.Dataset>
Dimensions:      (time: 1, x: 40, y: 45)
Coordinates:
  * time         (time) datetime64[ns] 2015-01-01
  * y            (y) float64 6.958e+06 6.958e+06 ... 6.947e+06 6.947e+06
  * x            (x) float64 5.296e+05 5.299e+05 ... 5.391e+05 5.394e+05
    spatial_ref  int32 32756
Data variables:
    blue         (time, y, x) int16 467 460 446 441 439 ... 431 446 441 418 448
    green        (time, y, x) int16 490 479 443 437 429 ... 516 542 510 481 499
    red          (time, y, x) int16 221 217 194 190 188 ... 324 350 272 246 261
    nir          (time, y, x) int16 85 83 83 81 83 83 84 ... 91 98 105 98 93 123
    swir1        (time, y, x) int16 51 47 48 46 48 47 47 ... 46 50 54 53 50 74
    swir2        (time, y, x) int16 41 38 39 37 39 38 39 ... 37 41 44 43 40 60
Attributes:
    crs:           EPSG:32756
    grid_mapping:  spatial_ref


Note that the `crs` attribute in the **Attributes** section has changed to `EPSG:32756`. 
Due to the larger 250 m resolution, there are now fewer pixels on the `x` and `y` dimensions (e.g. `x: 40, y: 45` compared to `x: 461, y: 508` in earlier examples).


### Spatial resampling methods
When a product is re-projected to a different CRS and/or resolution, the new pixel grid may differ from the original input pixels by size, number and alignment.
It is therefore necessary to apply a spatial "resampling" rule that allocates input pixel values into the new pixel grid.

By default, `dc.load()` resamples pixel values using "nearest neighbour" resampling, which allocates each new pixel with the value of the closest input pixel.
Depending on the type of data and the analysis being run, this may not be the most appropriate choice (e.g. for continuous data).

The `resampling` parameter in `dc.load()` allows you to choose a custom resampling method from the following options: 

```
"nearest", "cubic", "bilinear", "cubic_spline", "lanczos", 
"average", "mode", "gauss", "max", "min", "med", "q1", "q3"
```

The example cell below requests that all loaded data is resampled using "average" resampling:

In [8]:
# Note the additional `resampling` parameter
ds_averageresampling = dc.load(product="ls8_nbart_geomedian_annual",
                               x=(153.3, 153.4),
                               y=(-27.5, -27.6),
                               time=("2015-01-01", "2015-12-31"),
                               resolution=(-250, 250),
                               resampling="average")

print(ds_averageresampling)


<xarray.Dataset>
Dimensions:      (time: 1, x: 47, y: 51)
Coordinates:
  * time         (time) datetime64[ns] 2015-01-01
  * y            (y) float64 -3.156e+06 -3.156e+06 ... -3.168e+06 -3.168e+06
  * x            (x) float64 2.067e+06 2.068e+06 ... 2.079e+06 2.079e+06
    spatial_ref  int32 3577
Data variables:
    blue         (time, y, x) int16 496 491 485 497 505 ... 667 438 462 292 328
    green        (time, y, x) int16 539 546 542 556 570 ... 827 508 548 511 480
    red          (time, y, x) int16 250 256 256 273 279 ... 730 361 411 373 413
    nir          (time, y, x) int16 109 96 90 98 95 91 ... 479 213 305 2810 2573
    swir1        (time, y, x) int16 65 54 50 56 54 55 ... 236 108 155 1227 1174
    swir2        (time, y, x) int16 53 44 40 45 43 45 ... 92 165 80 106 574 558
Attributes:
    crs:           EPSG:3577
    grid_mapping:  spatial_ref


Python dictionaries can be used to request different sampling methods for different measurements. 
This can be particularly useful when some measurements contain contain categorical data which require resampling methods such as "nearest" or "mode" that do not modify the input pixel values.

The example cell below specifies `resampling={"red": "nearest", "*": "average"}`, which implements "nearest" neighbour resampling for the `red` satellite band only. `"*": "average"` will apply "average" resampling for all other satellite bands:


In [9]:
ds_customresampling = dc.load(product="ls8_nbart_geomedian_annual",
                              x=(153.3, 153.4),
                              y=(-27.5, -27.6),
                              time=("2015-01-01", "2015-12-31"),
                              output_crs="EPSG:32756",
                              resolution=(-250, 250),
                              resampling={"red": "nearest", "*": "average"})

print(ds_customresampling)

<xarray.Dataset>
Dimensions:      (time: 1, x: 40, y: 45)
Coordinates:
  * time         (time) datetime64[ns] 2015-01-01
  * y            (y) float64 6.958e+06 6.958e+06 ... 6.947e+06 6.947e+06
  * x            (x) float64 5.296e+05 5.299e+05 ... 5.391e+05 5.394e+05
    spatial_ref  int32 32756
Data variables:
    blue         (time, y, x) int16 464 455 447 445 442 ... 440 440 428 426 433
    green        (time, y, x) int16 482 463 446 444 436 ... 524 515 492 487 489
    red          (time, y, x) int16 221 217 194 190 188 ... 324 350 272 246 261
    nir          (time, y, x) int16 84 83 83 82 82 83 ... 94 100 100 95 102 115
    swir1        (time, y, x) int16 49 48 48 47 46 47 47 ... 48 52 54 51 57 67
    swir2        (time, y, x) int16 40 38 38 38 38 38 39 ... 38 43 44 41 46 54
Attributes:
    crs:           EPSG:32756
    grid_mapping:  spatial_ref


> For more information about spatial resampling methods, see the [following guide](https://rasterio.readthedocs.io/en/stable/topics/resampling.html)

## Loading data using the query dictionary syntax
It is often useful to re-use a set of query parameters to load data from multiple products.
To achieve this, load data using the "query dictionary" syntax.
This involves placing the query parameters inside a Python dictionary object which can be re-used for multiple data loads:

In [10]:
query = {"x": (153.3, 153.4),
         "y": (-27.5, -27.6),
         "time": ("2015-01-01", "2015-12-31")}


The query dictionary object can be added as an input to `dc.load()`. 

> The `**` syntax below is Python's "keyword argument unpacking" operator.
This operator takes the named query parameters listed in the query dictionary (e.g. `"x": (153.3, 153.4)`), and "unpacks" them into the `dc.load()` function as new arguments. 
For more information about unpacking operators, refer to the [Python documentation](https://docs.python.org/3/tutorial/controlflow.html#unpacking-argument-lists)

In [11]:
ds = dc.load(product="ls8_nbart_geomedian_annual",
             **query)

print(ds)

<xarray.Dataset>
Dimensions:      (time: 1, x: 461, y: 508)
Coordinates:
  * time         (time) datetime64[ns] 2015-01-01
  * y            (y) float64 -3.156e+06 -3.156e+06 ... -3.168e+06 -3.168e+06
  * x            (x) float64 2.067e+06 2.067e+06 ... 2.079e+06 2.079e+06
    spatial_ref  int32 3577
Data variables:
    blue         (time, y, x) int16 491 490 489 493 499 ... 372 313 289 286 294
    green        (time, y, x) int16 534 538 538 543 548 ... 545 483 421 394 399
    red          (time, y, x) int16 248 252 252 253 257 ... 457 364 347 362 350
    nir          (time, y, x) int16 108 106 105 105 108 ... 2522 2292 2351 2399
    swir1        (time, y, x) int16 64 65 62 62 64 ... 1616 1261 1057 1070 1101
    swir2        (time, y, x) int16 53 53 51 51 54 51 ... 886 627 510 503 512
Attributes:
    crs:           EPSG:3577
    grid_mapping:  spatial_ref


Query dictionaries can contain any set of parameters that would usually be provided to `dc.load()`:

In [12]:
query = {"x": (153.3, 153.4),
         "y": (-27.5, -27.6),
         "time": ("2015-01-01", "2015-12-31"),
         "output_crs": "EPSG:32756",
         "resolution": (-250, 250)}

ds_ls8 = dc.load(product="ls8_nbart_geomedian_annual",
                 **query)

print(ds_ls8)


<xarray.Dataset>
Dimensions:      (time: 1, x: 40, y: 45)
Coordinates:
  * time         (time) datetime64[ns] 2015-01-01
  * y            (y) float64 6.958e+06 6.958e+06 ... 6.947e+06 6.947e+06
  * x            (x) float64 5.296e+05 5.299e+05 ... 5.391e+05 5.394e+05
    spatial_ref  int32 32756
Data variables:
    blue         (time, y, x) int16 467 460 446 441 439 ... 431 446 441 418 448
    green        (time, y, x) int16 490 479 443 437 429 ... 516 542 510 481 499
    red          (time, y, x) int16 221 217 194 190 188 ... 324 350 272 246 261
    nir          (time, y, x) int16 85 83 83 81 83 83 84 ... 91 98 105 98 93 123
    swir1        (time, y, x) int16 51 47 48 46 48 47 47 ... 46 50 54 53 50 74
    swir2        (time, y, x) int16 41 38 39 37 39 38 39 ... 37 41 44 43 40 60
Attributes:
    crs:           EPSG:32756
    grid_mapping:  spatial_ref


After specifying the reusable query, it can be easily used to load data from a different product.
The example cell below loads Landsat 7 data for the same extent, time, output CRS and resolution as the previously loaded Landsat 8 data:

In [13]:
ds_ls7 = dc.load(product="ls7_nbart_geomedian_annual",
                 **query)

print(ds_ls7)

<xarray.Dataset>
Dimensions:      (time: 1, x: 40, y: 45)
Coordinates:
  * time         (time) datetime64[ns] 2015-01-01
  * y            (y) float64 6.958e+06 6.958e+06 ... 6.947e+06 6.947e+06
  * x            (x) float64 5.296e+05 5.299e+05 ... 5.391e+05 5.394e+05
    spatial_ref  int32 32756
Data variables:
    blue         (time, y, x) int16 492 447 420 419 417 ... 477 475 511 430 438
    green        (time, y, x) int16 513 462 426 419 414 ... 561 553 575 505 496
    red          (time, y, x) int16 269 236 211 209 206 ... 409 400 371 302 295
    nir          (time, y, x) int16 168 152 138 148 145 ... 204 208 230 172 188
    swir1        (time, y, x) int16 90 84 81 85 86 86 ... 103 99 106 139 94 109
    swir2        (time, y, x) int16 79 76 72 79 78 77 74 ... 93 90 95 122 82 95
Attributes:
    crs:           EPSG:32756
    grid_mapping:  spatial_ref


## Other helpful tricks
### Loading data "like" another dataset
Another option for loading matching data from multiple products is to use `dc.load()`'s `like` parameter.
This will copy the spatial and temporal extent and the CRS/resolution from an existing dataset, and use these parameters to load new data from a new product.

The example cell below loads another Landsat 7 dataset that exactly matches the `ds_ls8` dataset loaded earlier:


In [14]:
ds_ls7 = dc.load(product="ls7_nbart_geomedian_annual",
                 like=ds_ls8)

print(ds_ls7)

<xarray.Dataset>
Dimensions:      (time: 1, x: 40, y: 45)
Coordinates:
  * time         (time) datetime64[ns] 2015-01-01
  * y            (y) float64 6.958e+06 6.958e+06 ... 6.947e+06 6.947e+06
  * x            (x) float64 5.296e+05 5.299e+05 ... 5.391e+05 5.394e+05
    spatial_ref  int32 32756
Data variables:
    blue         (time, y, x) int16 492 447 420 419 417 ... 477 475 511 430 438
    green        (time, y, x) int16 513 462 426 419 414 ... 561 553 575 505 496
    red          (time, y, x) int16 269 236 211 209 206 ... 409 400 371 302 295
    nir          (time, y, x) int16 168 152 138 148 145 ... 204 208 230 172 188
    swir1        (time, y, x) int16 90 84 81 85 86 86 ... 103 99 106 139 94 109
    swir2        (time, y, x) int16 79 76 72 79 78 77 74 ... 93 90 95 122 82 95
Attributes:
    crs:           PROJCS["WGS 84 / UTM zone 56S",GEOGCS["WGS 84",DATUM["WGS_...
    grid_mapping:  spatial_ref


### Adding a progress bar
When loading large amounts of data, it can be useful to view the progress of the data load. 
The `progress_cbk` parameter in `dc.load()` adds a progress bar that indicates how the load is progressing:

![Progress bar](../Supplementary_data/04_Loading_data/progress_bar.jpg)

The example cell below loads 5 years of data (2013, 2014, 2015, 2016 and 2017) from the `ls8_nbart_geomedian_annual` product with a progress bar:

In [15]:
query = {"x": (153.3, 153.4),
         "y": (-27.5, -27.6),
         "time": ("2013", "2017")}

ds_progress = dc.load(product="ls8_nbart_geomedian_annual",
                      progress_cbk=with_ui_cbk(),
                      **query)

print(ds_progress)

<xarray.Dataset>
Dimensions:      (time: 5, x: 461, y: 508)
Coordinates:
  * time         (time) datetime64[ns] 2013-01-01 2014-01-01 ... 2017-01-01
  * y            (y) float64 -3.156e+06 -3.156e+06 ... -3.168e+06 -3.168e+06
  * x            (x) float64 2.067e+06 2.067e+06 ... 2.079e+06 2.079e+06
    spatial_ref  int32 3577
Data variables:
    blue         (time, y, x) int16 516 517 523 514 506 ... 360 331 310 300 314
    green        (time, y, x) int16 552 554 565 553 542 ... 531 472 408 389 416
    red          (time, y, x) int16 268 273 282 272 265 ... 420 363 344 348 355
    nir          (time, y, x) int16 108 112 116 106 105 ... 2615 2446 2494 2644
    swir1        (time, y, x) int16 63 63 68 60 61 ... 1510 1163 964 1004 1127
    swir2        (time, y, x) int16 51 50 53 49 49 50 ... 730 541 453 449 499
Attributes:
    crs:           EPSG:3577
    grid_mapping:  spatial_ref


## Recommended next steps

To continue working through the notebooks in this beginner's guide, the following notebooks are designed to be worked through in the following order:

1. [Jupyter Notebooks](01_Jupyter_notebooks.ipynb)
2. [Digital Earth Australia](02_DEA.ipynb)
3. [Products and measurements](03_Products_and_measurements.ipynb)
4. **Loading data (this notebook)**
5. [Plotting](05_Plotting.ipynb)
6. [Performing a basic analysis](06_Basic_analysis.ipynb)
7. [Introduction to Numpy](07_Intro_to_numpy.ipynb)
8. [Introduction to Xarray](08_Intro_to_xarray.ipynb)
9. [Parallel processing with Dask](09_Parallel_processing_with_Dask.ipynb)

Once you have worked through the beginner's guide, you can join advanced users by exploring:

* A demonstration of how to load cloud-free observations in the [using load_ard](../Frequently_used_code/Using_load_ard.ipynb) notebook.
* The "DEA datasets" directory in the repository, where you can explore DEA products in depth.
* The "Frequently used code" directory, which contains a recipe book of common techniques and methods for analysing DEA data.
* The "Real-world examples" directory, which provides more complex workflows and analysis case studies.

***
## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Australia data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to report an issue with this notebook, you can file one on [Github](https://github.com/GeoscienceAustralia/dea-notebooks).

**Last modified:** October 2020

**Compatible datacube version:** 

In [16]:
print(datacube.__version__)

1.8.3


## Tags
Browse all available tags on the DEA User Guide's [Tags Index](https://docs.dea.ga.gov.au/genindex.html)